In [169]:
import pandas as pd
import os 
import json

def read_csv(name):
    """"read a file and convert it to a pd dataframe"""
    files = os.listdir("./")
    for file in files:
        # file_name = file.rsplit('.',1)[0]
        if file.endswith(".csv") and file == name:
            raw_df = pd.read_csv(os.path.join("./",file), header= None)
            header = ["SegmentNr", "Position", "A", "C", "G", "T"]
            raw_df.columns = header
            raw_df = raw_df.reset_index(drop = True)
            k = file.rsplit('.',1)[0][-1]
            k = int(k)
            return raw_df, k, name
    else:
        raise LookupError("This file is not existing")
            
    



In [170]:
def clean_data(data_frame):
        # cleaning step1: ( number of positions are less than max of the position)
        # redundant data with similar data
        data_frame = data_frame.drop_duplicates()
        
        cl1 = data_frame.groupby("SegmentNr").agg({"Position" : ["max", "nunique"]})
        cl1 = cl1.reset_index()
        error_data = cl1[cl1["Position"]["max"] > cl1["Position"]["nunique"]]["SegmentNr"]
        to_del = data_frame[data_frame["SegmentNr"].isin(error_data)].index
        data_frame = data_frame.drop(to_del)

        # cleaning step2:

        cl2 = data_frame.groupby(["SegmentNr", "Position"]).agg({"A": ["count","max", "min"],\
                "C": ["max", "min"], "G": ["max", "min"],\
                        "T": ["max", "min"]}).reset_index()
        cl2.columns = [''.join(col).strip() for col in cl2.columns.values]

        # redundant data
        cl2 = cl2[cl2["Acount"] > 1]

        # redundant data with different values
        cl_diff = cl2[((cl2["Amax"] != cl2["Amin"]) | 
                (cl2["Cmax"] != cl2["Cmin"]) | 
                (cl2["Gmax"] != cl2["Gmin"]) | 
                (cl2["Tmax"] != cl2["Tmin"]))]
        to_del = data_frame[data_frame["SegmentNr"].isin(cl_diff["SegmentNr"])].index
        data_frame = data_frame.drop(to_del)

        # cleaning step 3:
        cl3 = data_frame.query("(A + C + G + T) != 1")
        to_del = data_frame[data_frame["SegmentNr"].isin(cl3["SegmentNr"])].index
        data_frame = data_frame.drop(to_del)
    
        # cleaning step 4:
        cl4 = data_frame.copy()
        cl4["Id"] = cl4[["Position", "A", "C", "G", "T"]].astype(str).agg(''.join, axis = 1)
        cl4 = cl4.groupby("SegmentNr").agg({"Id": lambda x: '\\'.join(x)}).reset_index()
        dups = cl4[cl4.duplicated("Id")]
        to_del =data_frame[data_frame["SegmentNr"].isin(dups["SegmentNr"])].index
        data_frame = data_frame.drop(to_del)
        data_frame = data_frame.reset_index(drop=True)
        return data_frame

    

In [171]:

df, k, filename = read_csv("DNA_3_3.csv")
df = clean_data(df)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [152]:
def generate_sequences(df):
    df["segment"] = df[['A', 'C', 'G', 'T']].idxmax(axis=1)
    df = df.sort_values(['SegmentNr', 'Position'])
    segment = df.groupby("SegmentNr").agg({"segment": lambda x: ''.join(x)}).reset_index()
    seg_json = segment.to_json(orient='records')
    seg_json = json.loads(seg_json)
    return seg_json


In [153]:
import networkx as nx 

df = generate_sequences(df)

In [108]:
# def construct_graph(json_data, k):
for i in df:
    print(i['segment'])

GGGT
TTGG
GTTT


In [154]:
def construct_graph(df, k):
    G = nx.MultiDiGraph()
    for i in df:
        seg = i['segment']
        steps = len(seg) - k
        k_mers = []
        for i in range(steps + 1):
            k_mers.append(seg[i:i+k])

        for i in k_mers:
            l_seg = i[:k-1]
            r_seg = i[1:k+1]
            G.add_edge(l_seg, r_seg)
    return G   


In [147]:
# df = construct_graph(df,k)
# list(df.edges(keys=False))

edges = [('ATTA', 'TTAC'), ('TTAC', 'TACT'), ('TACT', 'ACTC'),
             ('ACTC', 'CTCG'), ('CTCG', 'TCGC'), ('TCGC', 'CGCT'), 
             ('CGCT', 'GCTA'), ('TCGC', 'GCTA')]
debruijn_graph = nx.MultiDiGraph()
for edge in edges:
    debruijn_graph.add_edge(edge[0], edge[1])


In [156]:
df

[{'SegmentNr': 1, 'segment': 'GGGT'},
 {'SegmentNr': 2, 'segment': 'TTGG'},
 {'SegmentNr': 3, 'segment': 'GTTT'}]

In [157]:
import matplotlib.pyplot as plt
grrr = construct_graph(df, k)        

def orthogonal_layout(G):
    pos = {}
    nodes = list(G.nodes)
    graph_size = int(len(nodes)**0.5) + (len(nodes) % int(len(nodes)**0.5) != 0)

    for i, node in enumerate(nodes):
        row = i // graph_size
        col = i % graph_size
        pos[node] = (col, row)
    return pos

def plot_graph(G, filename):
    fig = plt.figure()
    pos = orthogonal_layout(G)
    
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightgreen')
    nx.draw_networkx_labels(G, pos, font_size=6, font_color='black', font_weight='bold')
    
    # for separating edges from each other
    for (u, v, key) in G.edges(keys=True):
        num_edges = G.number_of_edges(u, v)
        index = list(G[u][v]).index(key)
        placeholder = (index - num_edges / 2) * 0.2
        rad = 0.2 + placeholder
        nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], connectionstyle=f'arc3,rad={rad}')
    plt.savefig(filename, format='png')
    plt.close(fig)
    
plot_graph(grrr, filename)



True

In [158]:
def is_connect(graph):
    visited = set()
    stack = []
    v = list(graph.nodes)[0]
    stack.append(v)
    while stack:
        v = stack.pop()
        if v not in visited:
            visited.add(v)
            neighbours = list(graph[v])
            neighbours.reverse()
            for w in neighbours:
                stack.append(w)
    if len(visited) < len(graph):
        return False            
    else:
        return True


def is_valid_graph(graph):    
    # check if graph is connected
    if not is_connect(graph):
        return False
    else:
        # check if number of nodes with diffent number of in and out degrees are 0 or 2
        # also check if there are 2 nodes in above list they sould be start and end points
        different_in_out = []
        for node in graph.nodes():
            if graph.out_degree(node) != graph.in_degree(node):
                different_in_out.append(graph.out_degree(node) - graph.in_degree(node))
        if len(different_in_out) not in [0,2]:
            return False
        elif len(different_in_out) == 2 and (different_in_out[0]\
            + different_in_out[1] != 0 or different_in_out[0]*different_in_out[1] != -1):
            return False                
        return True
            
        
    

In [159]:
is_valid_graph(grrr)

True

In [165]:
def Eulerian_path_gen(graph):
    start_node =''
    for node in graph.nodes():
        if graph.out_degree(node) - graph.in_degree(node) == 1:
            start_node = node
            break
    if not start_node:
        for node in graph.nodes():
            if graph.out_degree(node) > 0:
                start_node = node
                break
    print(f"start from node{start_node}")
    G = graph.copy()
    stack = []
    path = []
    v = start_node
    stack.append(v)
    out_edges = dict(G.out_degree())
    while stack:
        v = stack[-1]
        if out_edges[v] == 0:
            if len(stack) > 1:
                u = stack[-2]
                path.insert(0,(u,v))
            stack.pop()
        else:
            # sort the edges based on name of the destination node
            edges = sorted(G.edges(v, keys=True), key=lambda x: x[1])
            for _, w, key in edges:
                stack.append(w)
                out_edges[v] -=1
                G.remove_edge(v, w, key=key)
                break
    return list(path)      

In [166]:
def construct_dna_sequence(graph):
    path = Eulerian_path_gen(graph)
    # path = list(nx.eulerian_path(graph))
    DNA = path[0][0]
    for i in path:
        DNA += i[1][-1]
    print(path)
    return DNA
    

In [168]:
DNA_edge_list = [('GG', 'GG'), ('GG', 'GT'), ('GT', 'TT'),('TT', 'TT'),
                 ('TT', 'TG'), ('TG', 'GG')]
debruijn_graph = nx.MultiDiGraph()
for edge in DNA_edge_list:
    debruijn_graph.add_edge(edge[0], edge[1])

construct_dna_sequence(debruijn_graph)


start from nodeGG
[('GG', 'GG'), ('GG', 'GT'), ('GT', 'TT'), ('TT', 'TT'), ('TT', 'TG'), ('TG', 'GG')]


'GGGTTTGG'

In [53]:
dna  = construct_dna_sequence(grrr)

[('TTAA', 'TAAT'), ('TAAT', 'AATT'), ('AATT', 'ATTA'), ('ATTA', 'TTAC'), ('TTAC', 'TACT'), ('TACT', 'ACTC'), ('ACTC', 'CTCA'), ('CTCA', 'TCAC'), ('TCAC', 'CACT'), ('CACT', 'ACTA'), ('ACTA', 'CTAC'), ('CTAC', 'TACG'), ('TACG', 'ACGC'), ('ACGC', 'CGCA'), ('CGCA', 'GCAC'), ('GCAC', 'CACT'), ('CACT', 'ACTG'), ('ACTG', 'CTGG'), ('CTGG', 'TGGC'), ('TGGC', 'GGCT'), ('GGCT', 'GCTA'), ('GCTA', 'CTAA'), ('CTAA', 'TAAT'), ('TAAT', 'AATT'), ('AATT', 'ATTA'), ('ATTA', 'TTAC'), ('TTAC', 'TACT'), ('TACT', 'ACTC'), ('ACTC', 'CTCA'), ('CTCA', 'TCAC'), ('TCAC', 'CACT'), ('CACT', 'ACTG'), ('ACTG', 'CTGG'), ('CTGG', 'TGGG'), ('TGGG', 'GGGT'), ('GGGT', 'GGTC'), ('GGTC', 'GTCA'), ('GTCA', 'TCAC'), ('TCAC', 'CACT'), ('CACT', 'ACTG')]


In [54]:
def save_output(s, filename):
    file = filename + ".txt"
    with open(file, 'w') as f:
        f.write(s)

In [55]:
save_output(dna, filename)

In [399]:
euler_path_nx = list(nx.eulerian_path(grrr))
for i in euler_path_nx:
    print(i)
    

('TTAA', 'TAAT')
('TAAT', 'AATT')
('AATT', 'ATTA')
('ATTA', 'TTAC')
('TTAC', 'TACT')
('TACT', 'ACTC')
('ACTC', 'CTCA')
('CTCA', 'TCAC')
('TCAC', 'CACT')
('CACT', 'ACTA')
('ACTA', 'CTAC')
('CTAC', 'TACG')
('TACG', 'ACGC')
('ACGC', 'CGCA')
('CGCA', 'GCAC')
('GCAC', 'CACT')
('CACT', 'ACTG')
('ACTG', 'CTGG')
('CTGG', 'TGGG')
('TGGG', 'GGGT')
('GGGT', 'GGTC')
('GGTC', 'GTCA')
('GTCA', 'TCAC')
('TCAC', 'CACT')
('CACT', 'ACTG')
('ACTG', 'CTGG')
('CTGG', 'TGGC')
('TGGC', 'GGCT')
('GGCT', 'GCTA')
('GCTA', 'CTAA')
('CTAA', 'TAAT')
('TAAT', 'AATT')
('AATT', 'ATTA')
('ATTA', 'TTAC')
('TTAC', 'TACT')
('TACT', 'ACTC')
('ACTC', 'CTCA')
('CTCA', 'TCAC')
('TCAC', 'CACT')
('CACT', 'ACTG')


In [429]:
grrr.edges('TAAT',keys=True)


OutMultiEdgeDataView([('TAAT', 'AATT', 0), ('TAAT', 'AATT', 1)])